# Logistic Regression Example - Predict the Success of Bank Telemarketing

Predict whether the client will subscribe a term deposit? This is a binary classification problem. The response variable, `y`, indicates if the client subscribed a term deposit and we'd like to predict it based on the historical data.

Data source:

[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. [A Data-Driven Approach to Predict the Success of Bank Telemarketing](http://media.salford-systems.com/video/tutorial/2015/targeted_marketing.pdf). Decision Support Systems, Elsevier, 62:22-31, June 2014

### Input variables
**Bank client data:**
1. age (numeric)
2. job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. default: has credit in default? (categorical: 'no','yes','unknown')
6. housing: has housing loan? (categorical: 'no','yes','unknown')
7. loan: has personal loan? (categorical: 'no','yes','unknown')

**Related with the last contact of the current campaign:**

8. contact: contact communication type (categorical: 'cellular','telephone') 
9. month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

**Other attributes:**

12. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. previous: number of contacts performed before this campaign and for this client (numeric)
15. poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

**Social and economic context attributes**

16. emp_var_rate: employment variation rate - quarterly indicator (numeric)
17. cons_price_idx: consumer price index - monthly indicator (numeric) 
18. cons_conf_idx: consumer confidence index - monthly indicator (numeric) 
19. euribor3m: euribor 3 month rate - daily indicator (numeric)
20. nr_employed: number of employees - quarterly indicator (numeric)

### Output variable (desired target)
21. y - has the client subscribed a term deposit? (binary: 'yes','no')

Importing libraries:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# the following line gets the bucket name attached to our cluster
bucket = spark._jsc.hadoopConfiguration().get("fs.gs.system.bucket")

# specifying the path to our bucket where the data is located (no need to edit this path anymore)
data = "gs://" + bucket + "/notebooks/jupyter/data/"
print(data)

Importing data:

In [ ]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load(data + "bank-additional-full.csv")\
  .coalesce(5)

df.cache()
df.printSchema()
print("This datasets consists of {} rows.".format(df.count()))

In [ ]:
pd.options.display.max_columns = None  # do not truncate the middle columns
df.limit(5).toPandas()

Define an RFormula that uses all of the 20 columns as features and call it `supervised`:

In [ ]:
# Your answer goes here

Fit the RFormula transformer and call it `fittedRF`:

In [ ]:
# Your answer goes here

Using `fittedRF` transform our `df` DataFrame. Call this `preparedDF`:

In [ ]:
# Your answer goes here

Print the first couple of rows of `preparedDF`, with the truncate option off:

In [ ]:
# Your answer goes here

Below we will retrieve the name of the columns used to make our feature vector and store them in a pandas DataFrame:

In [ ]:
featureCols = pd.DataFrame(preparedDF.schema["features"].metadata["ml_attr"]["attrs"]["binary"]+
  preparedDF.schema["features"].metadata["ml_attr"]["attrs"]["numeric"]).sort_values("idx")

featureCols = featureCols.set_index('idx')
featureCols.head()

Split the transformed data into `train` and `test`. Use a 30% split and a `seed = 843`.

In [ ]:
# Your answer goes here

Instantiate an instance of `LogisticRegression`. Call it `lr`:

In [ ]:
# Your answer goes here

Print the parameters of `lr` to check the default values used. You can always come back to the cell above and change the default values:

In [ ]:
# Your answer goes here

Fit the model on `train` and call it `lrModel`:

In [ ]:
# Your answer goes here

Below we plot the coefficients of our model in a sorted fashion:

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)

In [ ]:
beta = np.sort(lrModel.coefficients)
plt.plot(beta)
plt.ylabel('Beta Coefficients')

### Feature importance

We already retrieved the name of the features. Let's join it with the coefficients to identify the ones with bigger absolute value:

In [ ]:
coefsArray = np.array(lrModel.coefficients)  # convert to np.array
coefsDF = pd.DataFrame(coefsArray, columns=['coefs'])  # to pandas

coefsDF = coefsDF.merge(featureCols, left_index=True, right_index=True)  # join it with featureCols we created above
coefsDF.sort_values('coefs', inplace=True)  # Sort them
coefsDF.head()

Plot a bar chart:

In [ ]:
plt.rcParams["figure.figsize"] = (20,3)

In [ ]:
plt.xticks(rotation=90)
plt.bar(coefsDF.name, coefsDF.coefs)
plt.title('Ranked coefficients from the logistic regression model')
plt.show()

From our fitted model, `lrModel`, extract the summary and call it `summary`:

In [ ]:
# Your answer goes here

From `summary` extract `areaUnderROC`. Note that this AUC is from the `train` dataset and we should pay more attention to the AUC coming from the `test` set:

In [ ]:
# Your answer goes here

From `summary` extract `roc` and convert it to a pandas DataFrame. Call it `roc`:

In [ ]:
# Your answer goes here

Visualize the `roc` DataFrame:

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)

In [ ]:
# Your answer goes here
roc.plot(x='FPR', y='TPR', style='-', legend=False)
plt.title('ROC Cruve')
plt.ylabel('TPR')

print('Train AUC:', summary.areaUnderROC)

Do the same with `pr` from `summary`:

In [ ]:
# Your answer goes here

Our baseline model looks promising. Let's do some predictions on the `test` set.

Pass the `test` set through our trained model. Called the resulting DataFrame `fittedTest`:

In [ ]:
# Your answer goes here

Print the first few rows of this DataFrame. Only show the following columns: "label", "prediction", "rawPrediction"

In [ ]:
# Your answer goes here

Make an evaluator from `BinaryClassificationEvaluator` function that calculates AUC. We will use this function to measure our model's performance on the `test` set. Call this evaluator `aucEvaluator`. 

Note that this function can be found under the `pyspark.ml.evaluation` module.

In [ ]:
# Your answer goes here

Using our `aucEvaluator` find out the AUC on the `test` set:

In [ ]:
# Your answer goes here

Great! We see that our test and train AUC's are very close. This shows that the model we created is generalizable and would perform well on unseen data.

### Next Steps

How can you improve this "baseline" model? We barely customized our model or features. Think about the ways you can improve the AUC. 

* Can you use pipeline API and create a grid search to tune the hyperparameters? 
* What are the hyperparameters that you would modify?
* Try different regularization techniques by changing `elasticNetParam`. How does it impact our prediction power?
